 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [65]:
# All imports in one place whether I need them or not
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [57]:
# Defining the X and y variables

X = df.drop(columns = 'made_donation_in_march_2007')
y = df['made_donation_in_march_2007']

## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [62]:
# Using accuracy score
majority_class = y.mode()[0]
y_pred = np.full(shape=y.shape, fill_value=majority_class)
accuracy_score(y, y_pred)

0.7620320855614974

In [89]:
# Using value_counts(normalize)
y.value_counts(normalize=True)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

recall score is the **True Pos**/**(True Pos + False Neg)**

In [71]:
recall_score(y, y_pred)

0.0

In [90]:
# This gives us the recall score in the report along with valuable other information
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.76      1.00      0.86       570
           1       0.00      0.00      0.00       178

   micro avg       0.76      0.76      0.76       748
   macro avg       0.38      0.50      0.43       748
weighted avg       0.58      0.76      0.66       748



/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The recall score here is zero.

## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [91]:
# split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, 
                                                    random_state=42, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((561, 4), (187, 4), (561,), (187,))

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [82]:
# Defining the selector and using the Standard Scaler

# selector = SelectKBest(f_classif)
pipeline = make_pipeline(
    StandardScaler(),
    SelectKBest(f_classif), 
    LogisticRegression(solver='lbfgs')
)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [88]:
param_grid = {
    'selectkbest__k': [1,2,3,4], 
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

# Five folds not the four I previously had.
gridsearch = GridSearchCV(pipeline, param_grid = param_grid, 
                          scoring = 'recall', cv = 5, 
                          return_train_score = True, verbose = 5)

gridsearch.fit(X_train, y_train)

# I keep getting an error for C. I've looked at documentation and examples and 
# I don't know how else to put it in the parameters.
# It actually wasn't my C value at all. Finally got it.

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=1 
[CV]  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel(n_jobs=1)]

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=2, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=2 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=2, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=3, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisti

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversion

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, selectkbest__k=4, score=0.0, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.7307692307692307, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.6153846153846154, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.8846153846153846, total=   0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=balanced, selectkb

...

[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.6153846153846154, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.8846153846153846, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.8846153846153846, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=1, score=0.8076923076923077, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=2 
[CV]  logisticregression__C=10000.0, logisticregression__cla

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversion

[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4, score=0.7307692307692307, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4, score=0.5384615384615384, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4, score=0.8461538461538461, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4 
[CV]  logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4, score=0.8461538461538461, total=   0.0s
[CV] logisticregression__C=10000.0, logisticregression__class_weight=balanced, selectkbest__k=4 
[CV]  logisticregression__C=10000.0, logisticregression__cla

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:    7.0s finished
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x112974840>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'selectkbest__k': [1, 2, 3, 4], 'logisticregression__class_weight': [None, 'balanced'], 'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=5)

In [92]:
pd.DataFrame(gridsearch.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
53,0.013732,0.005798,0.003243,0.000953,100,balanced,2,"{'logisticregression__C': 100.0, 'logisticregr...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.788462,0.784615,0.013043
20,0.010158,0.002863,0.003022,0.000832,0.01,balanced,1,"{'logisticregression__C': 0.01, 'logisticregre...",0.730769,0.615385,...,0.784519,1.019843e-01,1,0.788462,0.817308,0.759615,0.759615,0.778846,0.780769,0.021414
38,0.008609,0.001060,0.002476,0.000549,1,balanced,3,"{'logisticregression__C': 1.0, 'logisticregres...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.788462,0.784615,0.013043
54,0.010640,0.002675,0.002412,0.000320,100,balanced,3,"{'logisticregression__C': 100.0, 'logisticregr...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.788462,0.784615,0.013043
46,0.009879,0.001514,0.002908,0.000935,10,balanced,3,"{'logisticregression__C': 10.0, 'logisticregre...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.788462,0.784615,0.013043
28,0.007840,0.000548,0.002433,0.000323,0.1,balanced,1,"{'logisticregression__C': 0.1, 'logisticregres...",0.730769,0.615385,...,0.784519,1.019843e-01,1,0.788462,0.817308,0.759615,0.759615,0.769231,0.778846,0.021926
29,0.009598,0.002558,0.002788,0.000649,0.1,balanced,2,"{'logisticregression__C': 0.1, 'logisticregres...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.778846,0.782692,0.013043
30,0.008875,0.001109,0.002270,0.000158,0.1,balanced,3,"{'logisticregression__C': 0.1, 'logisticregres...",0.730769,0.615385,...,0.784519,9.904613e-02,1,0.807692,0.778846,0.769231,0.778846,0.778846,0.782692,0.013043
12,0.013475,0.003256,0.007015,0.006449,0.001,balanced,1,"{'logisticregression__C': 0.001, 'logisticregr...",0.730769,0.615385,...,0.784519,1.019843e-01,1,0.798077,0.817308,0.759615,0.759615,0.778846,0.782692,0.022427
52,0.017172,0.001832,0.004857,0.001475,100,balanced,1,"{'logisticregression__C': 100.0, 'logisticregr...",0.730769,0.615385,...,0.784519,1.019843e-01,1,0.788462,0.817308,0.750000,0.759615,0.769231,0.776923,0.023865


## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [102]:
results = pd.DataFrame(gridsearch.cv_results_)
print(f'Best result from grid search of {len(results)} parameter combinations', '\n')
print('Best Cross-Validation Score: ',gridsearch.best_score_, '\n')
print('The values of C, the class_weight,  and k are:')
gridsearch.best_params_

Best result from grid search of 72 parameter combinations 

Best Cross-Validation Score:  0.784519402166461 

The values of C, the class_weight,  and k are:


{'logisticregression__C': 0.0001,
 'logisticregression__class_weight': 'balanced',
 'selectkbest__k': 1}

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [46]:
# True Pos plus True Neg over the total
(36+85)/187

0.6470588235294118

Calculate precision

In [47]:
# True Pos over true pos plus false pos
36/94

0.3829787234042553

Calculate recall

In [48]:
# True Pos over over True Pos plus False Neg
36/(36+8)

0.8181818181818182

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 